In [13]:
# Import pour la génération aléatoire
import random as random
# Import pour l'affichage uniquement
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
# Import pour les tests de mesure temporelle
import time


class Generateur(object):
    """ Générateur de mot plausible implémentant un dictionnaire de dictionnaire
    """

    def __init__(self):
        self.nbAppear = dict() # Initialise un dictionnaire vide
        
    def ajout(self, caractere1,caractere2):
        """ Incrémente le nombre d'occurence d'un couple de lettre, le créé si non présent
        """
        if caractere1 in self.nbAppear : # On vérifie la présence du premier caractère
            if caractere2 in self.nbAppear[caractere1] : # On vérifie la présence du second caractère
                self.nbAppear[caractere1][caractere2] += 1 # On l'incrémente s'il existe
            else :
                self.nbAppear[caractere1][caractere2] = 1 # On créé l'entrée sinon
        else :
            self.nbAppear[caractere1] = {caractere2 : 1} # On créé le sous dictionnaire avec l'entrée
        
    def lireMot(self, mot):
        """ Lit un mot et dénombre les occurrences de ses couple de lettre
        """
        for i in range(len(mot)-1): # On parcourt le mot
            self.ajout(mot[i],mot[i+1]) # On incrémente le nombre d'occurrence du couple

    def __estLettre(self, c):
        """ Détermine si un caractère donné est une lettre
        """
        return ("a" <= c and c <= "z") or ("à" <= c and c <= "ü")

    def __trouveMot(self,texte,i):
        """ Trouve le premier mot dans une chaîne donnée  partir d'un indice donné.
            Renvoi le mot et l'indice de fin
        """
        mot = " " # On démarre par le caractère de début de mot
        while self.__estLettre(texte[i].lower()) and i < len(texte): # On parcourt tant qu'on a une lettre
            mot+=texte[i].lower() # On ajoute la lettre au mot
            i+=1
        return mot+" ", i+1 # On ajoute une espace comme caractère de fin de mot

    def lireTexte(self,fichier):
        """ Lit un fichier texte dont le nom est donné en paramètre pour nourrir le dictionnaire
        """
        texte = open("./ProjetPython/"+fichier, "r").read()
        i = 0
        while i < len(texte):
            mot, i = self.__trouveMot(texte,i) # On récupère les mots
            if mot != "  " : # On nourri le dictionnaire si le mot n'est pas vide
                self.lireMot(mot)
    
    def __trouveLettre(self,caractere):
        """ Fait la somme des occurrences des lettres possibles suivant une lettre donnée 
            et fait un tirage uniforme sur cette somme
        """
        somm = 0
        for occurrence in self.nbAppear[caractere].values(): # Pour toutes les occurrences dans le dictionnaire d'une lettre
            somm += occurrence # On l'ajoute à la somme
        return random.randint(0,somm) # On renvoi une valeur aléatoire entre 0 et la somme
        
    def caractereSuivant(self, caractere):
        """ Prend un caractère et tire au sort le caractère suivant
        """
        alea = self.__trouveLettre(caractere) # On fait un tirage aléatoire
        i = 0
        occurences = list(self.nbAppear[caractere].values())
        while alea > occurences[i]: # Si le tirage est plus grand que le nombre d'occurence d'une lettre
            alea-= occurences[i] # On soustrait au tirage le nombre d'occurences de cette lettre
            i+=1
        return list(self.nbAppear[caractere].keys())[i] # On renvoi la lettre tiré
    
    def genereMot(self, premiereLettre=' '):
        """ Génère un mot commançant par une lettre donnée. Trouve la première lettre suivant le caractère de début de mot si non spécifié
        """
        mot = premiereLettre
        while(mot[-1]!=' ' or len(mot)==1): # Tant que la dernière lettre n'est pas le caractère de fin de mot ou que le mot est vide
            mot+=self.caractereSuivant(mot[-1]) # On ajoute la lettre suivante en fonction de la dernière lettre actuelle
        return mot[1:] if mot[0]==' ' else mot # Renvoi la mot, sans espace si première lettre non spécifiée

    def genereMotFixe(self, taille, premiereLettre=' '):
        """ Génère un mot commançant par une lettre donnée et d'une taille donnée. Trouve la première lettre suivant le caractère de début de mot si non spécifié"""
        mot = premiereLettre
        i = 0
        boucle = 0
        while(i < (taille-1 if mot[0]!=' ' else taille) and boucle<100): # Tant que le mot n'a pas ateint la taille spécifiée ou que la boucle de fin de mot à trop itérée
            l = self.caractereSuivant(mot[-1])
            boucle = 0
            while(l==' ' and boucle<100): # Si le caractère de fin de mot arrive trop tôt et n'est pas trop courant
                l = self.caractereSuivant(mot[-1]) # On recommance
                boucle+=1
            mot+=l
            i+=1
        return mot[1:] if mot[0]==' ' else mot
    
    def afficheMatrice(self):
        """ Génère la matrice de transition et l'affiche
        """
        ToutesLettres = sorted([key for key in self.nbAppear.keys()]) # On récupère toute les lettres
        # On génère une matrice carrée numpy en fonction du nombre de lettre
        prop = np.matrix([[0. for y in range(0,len(ToutesLettres))] for x in range(0,len(ToutesLettres))])
        
        
        # On remplit cette matrice avec les occurrences dans le dictionnaire
        i = 0
        for letter1 in ToutesLettres:
            y = 0
            for letter2 in ToutesLettres:
                if letter2 in self.nbAppear[letter1].keys():
                    prop[i,y] = self.nbAppear[letter1][letter2]
                y+=1
            i+=1
        
        # Transforme les occurences en pourcentage
        Somm = [x.sum() for x in prop]
        for i in range(0,len(Somm)) :
            for y in range(0,len(Somm)) :
                prop[i,y] = (float(prop[i,y])*100.0)/float(Somm[i])
        
        # Affiche la matrice
        fig, ax = plt.subplots(figsize=(7,7))
        im = ax.imshow(prop, cmap=cm.binary)

        ax.set_xticks(np.arange(len(ToutesLettres)))
        ax.set_yticks(np.arange(len(ToutesLettres)))

        ax.set_xticklabels(ToutesLettres)
        ax.set_yticklabels(ToutesLettres)

        ax.set_title("Probability of letters")
        fig.tight_layout()
        plt.show()
    
    
g = Generateur()
débutLecture = time.time()
g.lireTexte("notredame.txt")
finLecture = time.time()

débutGeneration = time.time()
for i in range(10):
    print("Mot fixe : ", g.genereMotFixe(10), "  Mot aléatoire : ", g.genereMot())
finGeneration = time.time()

print(" Temps de construction : ", finLecture - débutLecture)
print(" Temps de génération   : ", finGeneration - débutGeneration)

Mot fixe :  leustoncar   Mot aléatoire :  e 
Mot fixe :  sstreletas   Mot aléatoire :  tés 
Mot fixe :  devesorile   Mot aléatoire :  s 
Mot fixe :  sectonetau   Mot aléatoire :  aunitegn 
Mot fixe :  cesaussoux   Mot aléatoire :  bous 
Mot fixe :  munelienti   Mot aléatoire :  aroure 
Mot fixe :  cuporonsis   Mot aléatoire :  éecis 
Mot fixe :  laillaueta   Mot aléatoire :  er 
Mot fixe :  açangesant   Mot aléatoire :  qu 
Mot fixe :  rcinsanomo   Mot aléatoire :  qu 
 Temps de construction :  1.311042308807373
 Temps de génération   :  0.004094362258911133
